In [76]:
import pandas as pd
import os
import statsmodels.formula.api as smf

In [97]:
bank_names = {
    "cb privatbank": "Privat_Bank",
    "credit agricole bank": "Credit_Agricole",
    "fuib": "FUIB",
    "kredobank": "Kredobank",
    "oschadbank": "Oschadbank",
    "otp bank": "OTP",
    "pivdennyi bank": "Pivdennyi",
    "raiffeisen bank": "Raiffeisen",
    "sense bank": "Sense",
    "ukrsibbank": "Ukrsibbank",
    "universal bank": "Universal"
}

In [116]:
data_folder = "./../../data/positive_banks"

inf_df = pd.read_csv(data_folder + "/INF.csv", index_col=0)
inf_df.index = pd.to_datetime(inf_df.index, format="%Y-%m-%d")
pr_df = pd.read_csv(data_folder + "/PR.csv", index_col=0)
pr_df.index = pd.to_datetime(pr_df.index, format="%Y-%m-%d")

stacked = {"Bank":[], "Date":[], "INF":[], "PR": []}
a = True
for i in [j for j in os.listdir(data_folder) if j not in ["INF.csv", "PR.csv"]]:
    var = i[:-4]
    stacked[var] = []
    df = pd.read_csv(f"{data_folder}/{i}", index_col=0)
    for date in df.index:
        for bank in df.columns:
            if a:
                stacked["Bank"].append(bank)
                stacked["Date"].append(date)
                stacked["INF"].append(inf_df[inf_df.index == date]["inf"].iloc[0])
                stacked["PR"].append(pr_df[pr_df.index == date]["PR"].iloc[0])
            stacked[var].append(df.loc[date][bank])
    a = False

# for i in stacked.keys():
#     print(i , len(stacked[i]))
resulting_df = pd.DataFrame(stacked).replace(bank_names)
dummies = pd.get_dummies(resulting_df.Bank, dtype=int)
dummies.columns = [f'is_{i}' for i in dummies]
for i in dummies:
    resulting_df[i] = dummies[i]

In [117]:
resulting_df.columns
resulting_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Bank                561 non-null    object 
 1   Date                561 non-null    object 
 2   INF                 561 non-null    float64
 3   PR                  561 non-null    float64
 4   CDR                 561 non-null    float64
 5   CR                  561 non-null    float64
 6   LAS                 561 non-null    float64
 7   NIA                 561 non-null    float64
 8   NIM                 561 non-null    float64
 9   OE                  561 non-null    float64
 10  RA                  561 non-null    float64
 11  ROA                 561 non-null    float64
 12  SCTA                561 non-null    float64
 13  SIZE                561 non-null    float64
 14  is_Credit_Agricole  561 non-null    int32  
 15  is_FUIB             561 non-null    int32  
 16  is_Kredo

In [124]:
regression_string = 'NIM ~ PR + CDR + INF + LAS + CR + NIA + OE + RA + ROA + SCTA + SIZE + \
    is_Credit_Agricole + is_FUIB + is_Kredobank + is_OTP + is_Oschadbank + is_Pivdennyi + \
        is_Privat_Bank + is_Raiffeisen + is_Sense + is_Ukrsibbank + is_Universal'

# regression_string = 'NIM ~ PR + CR + LAS + SIZE + \
#     is_Credit_Agricole + is_FUIB + is_Kredobank + is_OTP + is_Oschadbank + is_Pivdennyi + \
#         is_Privat_Bank + is_Raiffeisen + is_Sense + is_Ukrsibbank + is_Universal'

mod3 = smf.ols(formula=regression_string, data=resulting_df)
res3 = mod3.fit()
print(res3.summary())

PatsyError: numbers besides '0' and '1' are only allowed with **
    NIM ~ PR + CDR + INF + LAS + CR + NIA + OE + RA + ROA + SCTA + SIZE +     is_Credit_Agricole + is_FUIB + is_Kredobank + is_OTP + is_Oschadbank + is_Pivdennyi +         is_Privat_Bank + is_Raiffeisen + is_Sense + is_Ukrsibbank + is_Universal
                     ^^^